In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xg
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from eli5.sklearn import PermutationImportance
from eli5 import show_weights
from sklearn.tree import DecisionTreeClassifier
from bayes_opt import BayesianOptimization

In [2]:
#Загрузим данные из файла.
df = pd.read_excel('/home/rik/Рабочий стол/МИФИ/учеба/мо/curse/data.xlsx', index_col=0)
df.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print("train.shape = {} rows, {} cols".format(*df.shape))

train.shape = 1001 rows, 213 cols


# Предобработка данных.

In [4]:
def transform(df): 
    discrete_feature = [feature for feature in df.columns if len(df[feature].unique())<25]
    count_discrete_feature = {}
    for i in tqdm(discrete_feature):
        cnt = 0
        for j in df[i]:
            if j != 0:
                cnt += 1
        count_discrete_feature[f'{i}'] = cnt
    count_discrete_feature = pd.Series(count_discrete_feature)
    df = df.drop(columns=count_discrete_feature[count_discrete_feature == 0].index.to_list())
    df = df.dropna()
    feature_del = ['BertzCT', 'Chi0', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
     'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan3', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1', 'Kappa2',
     'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MolMR', 'MolWt', 'NHOHCount', 'NumAromaticCarbocycles',
     'NumHAcceptors', 'NumHeteroatoms', 'NumSaturatedCarbocycles', 'NumValenceElectrons', 'SMR_VSA4', 'SMR_VSA9',
     'SlogP_VSA6', 'TPSA', 'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'fr_Al_OH_noTert', 'fr_COO', 'fr_COO2',
     'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond',
    'MinAbsPartialCharge', 'Chi2n']
    df = df.drop(columns=feature_del)
    df =  df[df['SI'] < 250]
    
    return df

In [5]:
df = transform(df)

100%|███████████████████████████████████████| 110/110 [00:00<00:00, 4323.21it/s]


In [6]:
val_median = df['SI'].median()

In [7]:
df['SI'] = df['SI'].apply(lambda x: 0 if x < val_median else 1)
df['SI'].head()

0    1
1    1
2    0
3    1
4    0
Name: SI, dtype: int64

## Обучение нескольких моделей, их сравнение. 

In [8]:
target = df['SI']
data = df.drop(['CC50, mM','IC50, mM','SI'], axis=1)

In [9]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 776 rows, 144 cols
x_valid.shape = 195 rows, 144 cols


In [10]:
# Создадим модель дерева решений
tree = DecisionTreeClassifier(random_state=27)
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# Создадим модель lightgbm
model_LGBM = LGBMClassifier(random_state=27)
# Создадим модель xgboost
xgb_r = xg.XGBClassifier() 
# обучение моделей
tree.fit(x_train, y_train)
model.fit(x_train, y_train, verbose=0)
model_LGBM.fit(x_train, y_train, verbose=0)
xgb_r.fit(x_train, y_train, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_tree_train = tree.predict(x_train)
y_pred_сat_Boost_train = model.predict(x_train)
y_pred_LGBM_train = model_LGBM.predict(x_train)
y_pred_xgb_train = xgb_r.predict(x_train)
# предсказание ответов для тестовой выборки
y_pred_tree = tree.predict(x_valid)
y_pred_сat_Boost = model.predict(x_valid)
y_pred_LGBM = model_LGBM.predict(x_valid)
y_pred_xgb = xgb_r.predict(x_valid)

print("Train tree f1_score: ", f1_score(y_train, y_pred_tree_train))
print("Train tree accuracy_score: ", accuracy_score(y_train, y_pred_tree_train))
print("Validation tree f1_score: ", f1_score(y_valid, y_pred_tree))
print("Validation tree accuracy_score: ", accuracy_score(y_valid, y_pred_tree))
print('*' * 50)
print("Train сat_Boost f1_score: ", f1_score(y_train, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid, y_pred_сat_Boost))
print('*' * 50)
print("Train LGBM f1_score: ", f1_score(y_train, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid, y_pred_LGBM))
print('*' * 50)
print("Train xgb f1_score: ", f1_score(y_train, y_pred_xgb_train))
print("Train xgb accuracy_score: ", accuracy_score(y_train, y_pred_xgb_train))
print("Validation xgb f1_score: ", f1_score(y_valid, y_pred_xgb))
print("Validation xgb accuracy_score: ", accuracy_score(y_valid, y_pred_xgb))

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train tree f1_score:  0.9461235216819974
Train tree accuracy_score:  0.9471649484536082
Validation tree f1_score:  0.5894736842105263
Validation tree accuracy_score:  0.6
**************************************************
Train сat_Boost f1_score:  0.905852417302799
Train сat_Boost accuracy_score:  0.904639175257732
Validation сat_Boost f1_score:  0.6494845360824741
Validation сat_Boost accuracy_score:  0.6512820512820513
**************************************************
Train LGBM f1_score:  0.9472329472329473
Train LGBM accuracy_score:  0.9471649484536082
Validation LGBM f1_score:  0.6564102564102564
Validation LGBM accuracy_score:  0.6564102564102564
**************************************************
Train xgb f1_score:  0.9475032010243278
Train xgb accuracy_score:  0.9471649484536082
Validation xgb f1_score:  0.606060606060606
Validation xgb accuracy_score:  0.6


Из приведенных данных выше мы видим низкое и среднее качество моделей, а также переобучение. Наилучшие результаты показали модели сat_Boost и LGBM. С ними и продолжим работать.

# Подбор признаков, их анализ и оценка важности.

**CatBoost**

In [11]:
col = x_train.columns.tolist()
estimator_cat_boost = PermutationImportance(
    model, scoring= "f1", random_state=27
)
importance_cat_boost = estimator_cat_boost.fit(x_train, y_train)

In [12]:
show_weights(importance_cat_boost, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0098 ± 0.0016,qed
0.0090 ± 0.0039,PEOE_VSA6
0.0086 ± 0.0014,MinAbsEStateIndex
0.0068 ± 0.0053,MinPartialCharge
0.0060 ± 0.0078,PEOE_VSA9
0.0054 ± 0.0057,EState_VSA4
0.0053 ± 0.0044,PEOE_VSA7
0.0047 ± 0.0040,VSA_EState1
0.0045 ± 0.0023,FpDensityMorgan2
0.0044 ± 0.0015,fr_aryl_methyl


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [13]:
feature_cat_boost = ['qed','PEOE_VSA6','MinAbsEStateIndex','MinPartialCharge','PEOE_VSA9','EState_VSA4','PEOE_VSA7',
'VSA_EState1','FpDensityMorgan2','fr_aryl_methyl','Ipc','BalabanJ','AvgIpc','EState_VSA5','EState_VSA6','EState_VSA7',
'VSA_EState5','VSA_EState10','fr_para_hydroxylation','VSA_EState9','Chi0n','SlogP_VSA12','PEOE_VSA3','NumAliphaticHeterocycles',
'PEOE_VSA8','VSA_EState4','SlogP_VSA4','EState_VSA3','fr_term_acetylene','fr_Ar_N','NOCount','NumSaturatedRings',
'SlogP_VSA1','fr_NH1','NumHDonors','PEOE_VSA5','fr_piperzine','EState_VSA2','SMR_VSA6','EState_VSA8','MaxEStateIndex',
'BCUT2D_LOGPHI','fr_halogen','fr_Ndealkylation1','SMR_VSA3','fr_methoxy','NumRotatableBonds','PEOE_VSA14',
'SlogP_VSA7','fr_NH0','fr_quatN','fr_pyridine','fr_sulfide','FractionCSP3','HallKierAlpha','BCUT2D_MWHI','PEOE_VSA13',
'SMR_VSA5','fr_aniline','NumSaturatedHeterocycles']

In [14]:
target = df['SI']
data_cb = data[feature_cat_boost]

In [15]:
x_train_cb, x_valid_cb = train_test_split(
    data_cb, train_size=0.8, random_state=1
)
y_train_cb, y_valid_cb = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_cb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_cb.shape))

x_train.shape = 776 rows, 60 cols
x_valid.shape = 195 rows, 60 cols


In [16]:
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# обучение моделей
model.fit(x_train_cb, y_train_cb, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_сat_Boost_train = model.predict(x_train_cb)
# предсказание ответов для тестовой выборки
y_pred_сat_Boost = model.predict(x_valid_cb)


print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_pred_сat_Boost))


Train сat_Boost f1_score:  0.9104859335038362
Train сat_Boost accuracy_score:  0.9097938144329897
Validation сat_Boost f1_score:  0.6632124352331605
Validation сat_Boost accuracy_score:  0.6666666666666666


Отобрав признаки мы улучшили метрики для модели CatBoost на вадидации f1_score: 0.66, accuracy_score: 0.67

**LGBM**

In [17]:
col = x_train.columns.tolist()
estimator_LGBM = PermutationImportance(
    model_LGBM, scoring= "f1", random_state=27
)
importance_LGBM = estimator_LGBM.fit(x_train, y_train)


In [18]:
show_weights(importance_LGBM, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0082 ± 0.0063,HallKierAlpha
0.0058 ± 0.0081,BCUT2D_MRLOW
0.0049 ± 0.0104,EState_VSA4
0.0047 ± 0.0108,BCUT2D_MRHI
0.0046 ± 0.0118,MaxEStateIndex
0.0042 ± 0.0046,MinPartialCharge
0.0039 ± 0.0055,MinEStateIndex
0.0036 ± 0.0090,NumRotatableBonds
0.0036 ± 0.0120,PEOE_VSA6
0.0032 ± 0.0095,PEOE_VSA7


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [19]:
feature_LGBM = ['HallKierAlpha','BCUT2D_MRLOW','EState_VSA4','BCUT2D_MRHI','MaxEStateIndex','MinPartialCharge',
'MinEStateIndex','NumRotatableBonds','PEOE_VSA6','PEOE_VSA7','PEOE_VSA9','BCUT2D_MWHI','PEOE_VSA8','VSA_EState4','SMR_VSA1',
'MaxPartialCharge','EState_VSA2','PEOE_VSA2','PEOE_VSA1','AvgIpc','EState_VSA9','Ipc','EState_VSA3','EState_VSA1',
'SlogP_VSA10','BCUT2D_LOGPHI','BalabanJ','fr_Ar_N','qed','fr_NH0','NumSaturatedRings','BCUT2D_LOGPLOW',
'NumAliphaticHeterocycles','VSA_EState5','NumHDonors','SMR_VSA6','MolLogP','NumSaturatedHeterocycles',
'fr_allylic_oxid','VSA_EState7','SlogP_VSA5','SMR_VSA10','PEOE_VSA10','PEOE_VSA4','Chi0n','VSA_EState9',
'BCUT2D_MWLOW','EState_VSA6','PEOE_VSA3','BCUT2D_CHGHI','NOCount','NumAliphaticRings','fr_C_S','EState_VSA7',
'PEOE_VSA13','VSA_EState10','fr_alkyl_carbamate','fr_ArN','fr_unbrch_alkane','fr_Ar_COO','fr_alkyl_halide','fr_aniline',
'fr_Al_COO','fr_bicyclic']

In [20]:
target_LGBM = df['SI']
data_LGBM = data[feature_LGBM]

In [21]:
x_train_LGBM, x_valid_LGBM = train_test_split(
    data_LGBM, train_size=0.8, random_state=1
)
y_train_LGBM, y_valid_LGBM = train_test_split(
    target_LGBM, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_LGBM.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_LGBM.shape))

x_train.shape = 776 rows, 64 cols
x_valid.shape = 195 rows, 64 cols


In [22]:
model_LGBM = LGBMClassifier(random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9470967741935484
Train LGBM accuracy_score:  0.9471649484536082
Validation LGBM f1_score:  0.6699507389162561
Validation LGBM accuracy_score:  0.6564102564102564


Отобрав признаки мы улучшили метрики для модели LGBMClassifier на вадидации f1_score: 0.67, accuracy_score: 0.66.

# Подбор гиперпараметров

**CatBoost**

In [23]:
def catboost_cv(depth, learning_rate,n_estimators):
    depth = int(depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(
        depth=depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_cb, y_train_cb, verbose=100)

    return f1_score(y_valid_cb, model.predict(x_valid_cb))

In [24]:
param_space = {
    'depth': (2, 10),             
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 1000)
}

bayesian_opt = BayesianOptimization(
    f=catboost_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=20)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |   depth   | learni... | n_esti... |
-------------------------------------------------------------
0:	learn: 0.5890805	total: 3.35ms	remaining: 2.5s
100:	learn: 0.9420849	total: 305ms	remaining: 1.95s
200:	learn: 0.9476373	total: 609ms	remaining: 1.66s
300:	learn: 0.9476373	total: 922ms	remaining: 1.37s
400:	learn: 0.9472329	total: 1.23s	remaining: 1.06s
500:	learn: 0.9477707	total: 1.52s	remaining: 752ms
600:	learn: 0.9473684	total: 1.82s	remaining: 445ms
700:	learn: 0.9479034	total: 2.12s	remaining: 142ms
747:	learn: 0.9479034	total: 2.26s	remaining: 0us
| 1         | 0.6392    | 5.406     | 0.2446    | 748.6     |
0:	learn: 0.6739130	total: 68.6ms	remaining: 1m 7s
100:	learn: 0.9450830	total: 1.76s	remaining: 15.3s
200:	learn: 0.9476373	total: 3.45s	remaining: 13.4s
300:	learn: 0.9473684	total: 5.1s	remaining: 11.5s
400:	learn: 0.9476373	total: 6.8s	remaining: 9.82s
500:	learn: 0.9476373	total: 8.57s	remaining: 8.2s
600:	learn: 0.9476373	total: 10.2s	remaini

200:	learn: 0.9475032	total: 990ms	remaining: 1.6s
300:	learn: 0.9475032	total: 1.47s	remaining: 1.09s
400:	learn: 0.9475032	total: 1.96s	remaining: 608ms
500:	learn: 0.9473684	total: 2.44s	remaining: 117ms
524:	learn: 0.9476373	total: 2.56s	remaining: 0us
| 16        | 0.6633    | 6.347     | 0.244     | 525.8     |
0:	learn: 0.6630872	total: 9.11ms	remaining: 4.11s
100:	learn: 0.9475032	total: 1.02s	remaining: 3.54s
200:	learn: 0.9476373	total: 1.89s	remaining: 2.37s
300:	learn: 0.9477707	total: 2.74s	remaining: 1.38s
400:	learn: 0.9479034	total: 3.58s	remaining: 456ms
451:	learn: 0.9479034	total: 4.03s	remaining: 0us
| 17        | 0.6468    | 7.069     | 0.1594    | 452.0     |
0:	learn: 0.5890805	total: 5.49ms	remaining: 2.63s
100:	learn: 0.9462916	total: 336ms	remaining: 1.26s
200:	learn: 0.9476373	total: 671ms	remaining: 928ms
300:	learn: 0.9475032	total: 1.06s	remaining: 629ms
400:	learn: 0.9476373	total: 1.37s	remaining: 267ms
478:	learn: 0.9479034	total: 1.62s	remaining: 0us
|

In [25]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'l2_leaf_reg'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'depth': 4, 'learning_rate': 0.1415804717945562, 'n_estimators': 775.6441966875759}
Best R-squared Score: 0.6736


In [26]:
params = {
    'depth': 4,             
    'learning_rate': 0.1415804717945562, 
    'n_estimators': 776
}
model = CatBoostClassifier(**params, random_state=27)
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_cb))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_cb))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_val_cb))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_val_cb))

0:	learn: 0.6819893	total: 3.23ms	remaining: 2.5s
100:	learn: 0.3492358	total: 207ms	remaining: 1.38s
200:	learn: 0.2267826	total: 411ms	remaining: 1.17s
300:	learn: 0.1719675	total: 646ms	remaining: 1.02s
400:	learn: 0.1404115	total: 861ms	remaining: 805ms
500:	learn: 0.1243704	total: 1.08s	remaining: 592ms
600:	learn: 0.1153113	total: 1.32s	remaining: 384ms
700:	learn: 0.1087095	total: 1.56s	remaining: 167ms
775:	learn: 0.1059704	total: 1.73s	remaining: 0us
Train сat_Boost f1_score:  0.9472329472329473
Train сat_Boost accuracy_score:  0.9471649484536082
Validation сat_Boost f1_score:  0.6735751295336788
Validation сat_Boost accuracy_score:  0.676923076923077


После подбора гиперпараметров мы улучшили метрики модели CatBoost на вадидации f1_score: 0.67, accuracy_score: 0.68.

**LGBM**

In [27]:
def LGBM_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = LGBMClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,          
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
    return f1_score(y_valid_LGBM, model.predict(x_valid_LGBM))

In [28]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 1000)
}

bayesian_opt = BayesianOptimization(
    f=LGBM_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=25)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.677618
[2]	valid_0's binary_logloss: 0.665184
[3]	valid_0's binary_logloss: 0.654325
[4]	valid_0's binary_logloss: 0.652278
[5]	valid_0's binary_logloss: 0.651161
[6]	valid_0's binary_logloss: 0.644638
[7]	valid_0's binary_logloss: 0.642489
[8]	valid_0's binary_logloss: 0.642485
[9]	valid_0's binary_logloss: 0.641966
[10]	valid_0's binary_logloss: 0.640132
[11]	valid_0's binary_logloss: 0.643624
[12]	valid_0's binary_logloss: 0.637071
[13]	valid_0's binary_logloss: 0.642737
[14]	valid_0's binary_logloss: 0.638316
[15]	valid_0's binary_logloss: 0.643204
[16]	valid_0's binary_logloss: 0.645225
[17]	valid_0's binary_logloss: 0.652573
[18]	valid_0's binary_logloss: 0.655003
[19]	valid_0's binary_logloss: 0.653293
[20]	valid_0's binary_logloss: 0.654444
[21]	valid_0's binary_logloss: 0.

[221]	valid_0's binary_logloss: 1.07344
[222]	valid_0's binary_logloss: 1.07468
[223]	valid_0's binary_logloss: 1.07504
[224]	valid_0's binary_logloss: 1.07589
[225]	valid_0's binary_logloss: 1.0791
[226]	valid_0's binary_logloss: 1.08199
[227]	valid_0's binary_logloss: 1.08352
[228]	valid_0's binary_logloss: 1.08711
[229]	valid_0's binary_logloss: 1.08634
[230]	valid_0's binary_logloss: 1.08919
[231]	valid_0's binary_logloss: 1.09242
[232]	valid_0's binary_logloss: 1.09501
[233]	valid_0's binary_logloss: 1.094
[234]	valid_0's binary_logloss: 1.09797
[235]	valid_0's binary_logloss: 1.1005
[236]	valid_0's binary_logloss: 1.1006
[237]	valid_0's binary_logloss: 1.10004
[238]	valid_0's binary_logloss: 1.10193
[239]	valid_0's binary_logloss: 1.10402
[240]	valid_0's binary_logloss: 1.11116
[241]	valid_0's binary_logloss: 1.11329
[242]	valid_0's binary_logloss: 1.11429
[243]	valid_0's binary_logloss: 1.11495
[244]	valid_0's binary_logloss: 1.11625
[245]	valid_0's binary_logloss: 1.12065
[246]

[513]	valid_0's binary_logloss: 1.43166
[514]	valid_0's binary_logloss: 1.43258
[515]	valid_0's binary_logloss: 1.43331
[516]	valid_0's binary_logloss: 1.43241
[517]	valid_0's binary_logloss: 1.43134
[518]	valid_0's binary_logloss: 1.43373
[519]	valid_0's binary_logloss: 1.43653
[520]	valid_0's binary_logloss: 1.4367
[521]	valid_0's binary_logloss: 1.43751
[522]	valid_0's binary_logloss: 1.43745
[523]	valid_0's binary_logloss: 1.43821
[524]	valid_0's binary_logloss: 1.43621
[525]	valid_0's binary_logloss: 1.43711
[526]	valid_0's binary_logloss: 1.43704
[527]	valid_0's binary_logloss: 1.43711
[528]	valid_0's binary_logloss: 1.43987
[529]	valid_0's binary_logloss: 1.44019
[530]	valid_0's binary_logloss: 1.44023
[531]	valid_0's binary_logloss: 1.44058
[532]	valid_0's binary_logloss: 1.44208
[533]	valid_0's binary_logloss: 1.44195
[534]	valid_0's binary_logloss: 1.44209
[535]	valid_0's binary_logloss: 1.44255
[536]	valid_0's binary_logloss: 1.4421
[537]	valid_0's binary_logloss: 1.44237
[5

[719]	valid_0's binary_logloss: 1.55324
[720]	valid_0's binary_logloss: 1.55308
[721]	valid_0's binary_logloss: 1.55286
[722]	valid_0's binary_logloss: 1.55341
[723]	valid_0's binary_logloss: 1.55348
[724]	valid_0's binary_logloss: 1.55273
[725]	valid_0's binary_logloss: 1.55429
[726]	valid_0's binary_logloss: 1.55478
[727]	valid_0's binary_logloss: 1.55507
[728]	valid_0's binary_logloss: 1.55666
[729]	valid_0's binary_logloss: 1.55716
[730]	valid_0's binary_logloss: 1.55846
[731]	valid_0's binary_logloss: 1.55932
[732]	valid_0's binary_logloss: 1.55967
[733]	valid_0's binary_logloss: 1.55971
[734]	valid_0's binary_logloss: 1.55994
[735]	valid_0's binary_logloss: 1.56049
[736]	valid_0's binary_logloss: 1.56064
[737]	valid_0's binary_logloss: 1.56082
[738]	valid_0's binary_logloss: 1.56065
[739]	valid_0's binary_logloss: 1.56091
[740]	valid_0's binary_logloss: 1.5611
[741]	valid_0's binary_logloss: 1.56053
[742]	valid_0's binary_logloss: 1.56075
[743]	valid_0's binary_logloss: 1.56083
[

[234]	valid_0's binary_logloss: 1.25006
[235]	valid_0's binary_logloss: 1.25331
[236]	valid_0's binary_logloss: 1.25288
[237]	valid_0's binary_logloss: 1.25448
[238]	valid_0's binary_logloss: 1.25669
[239]	valid_0's binary_logloss: 1.25882
[240]	valid_0's binary_logloss: 1.26478
[241]	valid_0's binary_logloss: 1.26218
[242]	valid_0's binary_logloss: 1.26038
[243]	valid_0's binary_logloss: 1.26204
[244]	valid_0's binary_logloss: 1.26188
[245]	valid_0's binary_logloss: 1.26249
[246]	valid_0's binary_logloss: 1.26371
[247]	valid_0's binary_logloss: 1.26232
[248]	valid_0's binary_logloss: 1.26333
[249]	valid_0's binary_logloss: 1.26479
[250]	valid_0's binary_logloss: 1.26588
[251]	valid_0's binary_logloss: 1.26727
[252]	valid_0's binary_logloss: 1.26857
[253]	valid_0's binary_logloss: 1.26977
[254]	valid_0's binary_logloss: 1.2688
[255]	valid_0's binary_logloss: 1.27215
[256]	valid_0's binary_logloss: 1.27384
[257]	valid_0's binary_logloss: 1.2742
[258]	valid_0's binary_logloss: 1.28113
[2

...

[239]	valid_0's binary_logloss: 0.641349
[240]	valid_0's binary_logloss: 0.641109
[241]	valid_0's binary_logloss: 0.641172
[242]	valid_0's binary_logloss: 0.641057
[243]	valid_0's binary_logloss: 0.641235
[244]	valid_0's binary_logloss: 0.641316
[245]	valid_0's binary_logloss: 0.641297
[246]	valid_0's binary_logloss: 0.641371
[247]	valid_0's binary_logloss: 0.641262
[248]	valid_0's binary_logloss: 0.641085
[249]	valid_0's binary_logloss: 0.641142
[250]	valid_0's binary_logloss: 0.641046
[251]	valid_0's binary_logloss: 0.641139
[252]	valid_0's binary_logloss: 0.64095
[253]	valid_0's binary_logloss: 0.641004
[254]	valid_0's binary_logloss: 0.640981
[255]	valid_0's binary_logloss: 0.640944
[256]	valid_0's binary_logloss: 0.640946
[257]	valid_0's binary_logloss: 0.64095
[258]	valid_0's binary_logloss: 0.640709
[259]	valid_0's binary_logloss: 0.640607
[260]	valid_0's binary_logloss: 0.640753
[261]	valid_0's binary_logloss: 0.64064
[262]	valid_0's binary_logloss: 0.640706
[263]	valid_0's bin

[537]	valid_0's binary_logloss: 0.653578
[538]	valid_0's binary_logloss: 0.653867
[539]	valid_0's binary_logloss: 0.653876
[540]	valid_0's binary_logloss: 0.653831
[541]	valid_0's binary_logloss: 0.653696
[542]	valid_0's binary_logloss: 0.653754
[543]	valid_0's binary_logloss: 0.653865
[544]	valid_0's binary_logloss: 0.654117
[545]	valid_0's binary_logloss: 0.65402
[546]	valid_0's binary_logloss: 0.654218
[547]	valid_0's binary_logloss: 0.654323
[548]	valid_0's binary_logloss: 0.654398
[549]	valid_0's binary_logloss: 0.65443
[550]	valid_0's binary_logloss: 0.65453
[551]	valid_0's binary_logloss: 0.654589
[552]	valid_0's binary_logloss: 0.654496
[553]	valid_0's binary_logloss: 0.654574
[554]	valid_0's binary_logloss: 0.654552
[555]	valid_0's binary_logloss: 0.65472
[556]	valid_0's binary_logloss: 0.654733
[557]	valid_0's binary_logloss: 0.654864
[558]	valid_0's binary_logloss: 0.654952
[559]	valid_0's binary_logloss: 0.655146
[560]	valid_0's binary_logloss: 0.655281
[561]	valid_0's bina

[809]	valid_0's binary_logloss: 0.669596
[810]	valid_0's binary_logloss: 0.669464
[811]	valid_0's binary_logloss: 0.669266
[812]	valid_0's binary_logloss: 0.669183
[813]	valid_0's binary_logloss: 0.669285
[814]	valid_0's binary_logloss: 0.669309
[815]	valid_0's binary_logloss: 0.669468
[816]	valid_0's binary_logloss: 0.669386
[817]	valid_0's binary_logloss: 0.669405
[818]	valid_0's binary_logloss: 0.669271
[819]	valid_0's binary_logloss: 0.66911
[820]	valid_0's binary_logloss: 0.669339
[821]	valid_0's binary_logloss: 0.669311
[822]	valid_0's binary_logloss: 0.669279
[823]	valid_0's binary_logloss: 0.66951
[824]	valid_0's binary_logloss: 0.669345
[825]	valid_0's binary_logloss: 0.669647
[826]	valid_0's binary_logloss: 0.669517
[827]	valid_0's binary_logloss: 0.66933
[828]	valid_0's binary_logloss: 0.669385
[829]	valid_0's binary_logloss: 0.669355
[830]	valid_0's binary_logloss: 0.66915
[831]	valid_0's binary_logloss: 0.669173
[832]	valid_0's binary_logloss: 0.669095
[833]	valid_0's bina

[317]	valid_0's binary_logloss: 0.922605
[318]	valid_0's binary_logloss: 0.924129
[319]	valid_0's binary_logloss: 0.924876
[320]	valid_0's binary_logloss: 0.926089
[321]	valid_0's binary_logloss: 0.926551
[322]	valid_0's binary_logloss: 0.927021
[323]	valid_0's binary_logloss: 0.928476
[324]	valid_0's binary_logloss: 0.929106
[325]	valid_0's binary_logloss: 0.931142
[326]	valid_0's binary_logloss: 0.932511
[327]	valid_0's binary_logloss: 0.933554
[328]	valid_0's binary_logloss: 0.936645
[329]	valid_0's binary_logloss: 0.936287
[330]	valid_0's binary_logloss: 0.937664
[331]	valid_0's binary_logloss: 0.939766
[332]	valid_0's binary_logloss: 0.939221
[333]	valid_0's binary_logloss: 0.939477
[334]	valid_0's binary_logloss: 0.939288
[335]	valid_0's binary_logloss: 0.938977
[336]	valid_0's binary_logloss: 0.940026
[337]	valid_0's binary_logloss: 0.940229
[338]	valid_0's binary_logloss: 0.942329
[339]	valid_0's binary_logloss: 0.942724
[340]	valid_0's binary_logloss: 0.944815
[341]	valid_0's 

[197]	valid_0's binary_logloss: 0.677613
[198]	valid_0's binary_logloss: 0.677437
[199]	valid_0's binary_logloss: 0.677827
[200]	valid_0's binary_logloss: 0.677765
[201]	valid_0's binary_logloss: 0.678065
[202]	valid_0's binary_logloss: 0.678115
[203]	valid_0's binary_logloss: 0.678035
[204]	valid_0's binary_logloss: 0.67799
[205]	valid_0's binary_logloss: 0.679022
[206]	valid_0's binary_logloss: 0.678545
[207]	valid_0's binary_logloss: 0.67903
[208]	valid_0's binary_logloss: 0.679476
[209]	valid_0's binary_logloss: 0.679432
[210]	valid_0's binary_logloss: 0.67898
[211]	valid_0's binary_logloss: 0.679662
[212]	valid_0's binary_logloss: 0.679636
[213]	valid_0's binary_logloss: 0.679238
[214]	valid_0's binary_logloss: 0.679343
[215]	valid_0's binary_logloss: 0.679589
[216]	valid_0's binary_logloss: 0.679803
[217]	valid_0's binary_logloss: 0.679768
[218]	valid_0's binary_logloss: 0.680543
[219]	valid_0's binary_logloss: 0.680669
[220]	valid_0's binary_logloss: 0.681397
[221]	valid_0's bin

In [29]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.0796548685784298, 'max_depth': 8.16293101479415, 'n_estimators': 159}
Best R-squared Score: 0.6765


In [30]:
params = {
    'max_depth': 8,            
    'learning_rate': 0.0796548685784298,  
    'n_estimators': 159
}

model_LGBM = LGBMClassifier(**params, random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9472329472329473
Train LGBM accuracy_score:  0.9471649484536082
Validation LGBM f1_score:  0.6764705882352942
Validation LGBM accuracy_score:  0.6615384615384615


После подбора гиперпараметров мы улучшили метрики и снизили переобучение для модели LGBMClassifier на вадидации f1_score: 0.676, accuracy_score: 0.661.

**Выводы**

Мы сравнили 4 регрессионые модели: DecisionTreeRegressor, CatBoostRegressor, LGBMRegressor, XGBRegressor. Наилучшие результаты показали модели LGBMClassifier с результатом на трейне: f1_score= 0.95 и accuracy_score= 0.95 и тесте: f1_score=0.676, accuracy_score=0.661 и XGBClassifier с результатом на трейне: f1_score= 0.95 и accuracy_score= 0.95 и тесте: f1_score=0.674, accuracy_score=0.677.

**Рекомендации**

Для улучшения значения метрики и снижения переобучения нужно будет увеличить количество данных.